In [9]:
import pandas as pd

In [14]:
df = pd.read_csv('/home/jovyan/work/dataset.csv', sep="\t", quotechar="'")
df_1 = pd.read_csv('/home/jovyan/work/dataset_1.csv', sep="\t", quotechar="'")

In [15]:
df.head()

,frame_number,frame_time_epoch,frame_len,eth_src,device_name_src,eth_dst,device_name_dst,eth_type,ip_src,srcport,ip_dst,dstport,ip_proto,payload_utf8,action,attack
0,1,1.674747e+09,66,04:CF:4B:3B:00:58,NaN,0E:F2:26:7D:04:1A,NaN,0x0800,18.196.142.136,8883.0,192.168.12.148,38462.0,6.0,NaN,NaN,NaN
1,2,1.674747e+09,303,A8:03:2A:B1:35:60,NaN,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.80,51705.0,34.140.193.14,6022.0,6.0,,NaN,NaN
2,3,1.674747e+09,66,0E:F2:26:7D:04:1A,NaN,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.148,38462.0,18.196.142.136,8883.0,6.0,NaN,NaN,NaN
3,4,1.674747e+09,54,04:CF:4B:3B:00:58,NaN,A8:03:2A:B1:35:60,NaN,0x0800,34.140.193.14,6022.0,192.168.12.80,51705.0,6.0,NaN,NaN,NaN
4,5,1.674747e+09,128,0E:F2:26:7D:04:1A,NaN,FF:FF:FF:FF:FF:FF,NaN,0x0800,192.168.12.148,52149.0,255.255.255.255,7000.0,17.0,NaN,NaN,NaN


In [16]:
df_1.head()

,frame_number,frame_time_epoch,frame_len,eth_src,device_name_src,eth_dst,device_name_dst,eth_type,ip_src,srcport,ip_dst,dstport,ip_proto,payload_utf8,action,attack
0,1,1.674748e+09,128,0E:F2:26:7D:04:1A,NaN,FF:FF:FF:FF:FF:FF,NaN,0x0800,192.168.12.148,49227.0,192.168.12.255,7000.0,17.0,NaN,NaN,NaN
1,2,1.674748e+09,128,0E:F2:26:7D:04:1A,NaN,FF:FF:FF:FF:FF:FF,NaN,0x0800,192.168.12.148,49227.0,192.168.12.255,7000.0,17.0,NaN,NaN,NaN
2,3,1.674748e+09,66,04:CF:4B:3B:00:58,NaN,0E:F2:26:7D:04:1A,NaN,0x0800,18.196.142.136,8883.0,192.168.12.148,38462.0,6.0,NaN,NaN,NaN
3,4,1.674748e+09,66,0E:F2:26:7D:04:1A,NaN,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.148,38462.0,18.196.142.136,8883.0,6.0,NaN,NaN,NaN
4,5,1.674748e+09,78,0E:F2:26:7D:04:1A,NaN,D8:F1:5B:D8:08:0C,LDR TUYA,0x0800,192.168.12.148,45590.0,192.168.12.113,6668.0,6.0,NaN,NaN,NaN


In [17]:
df.shape, df_1.shape

((2633, 16), (2335, 16))

In [18]:
df.describe()

,frame_number,frame_time_epoch,frame_len,srcport,dstport,ip_proto
count,2633.000000,2.633000e+03,2633.000000,1486.000000,1486.000000,1386.000000
mean,1317.000000,1.674747e+09,139.031143,22048.778600,17192.418573,7.634921
std,760.225953,2.402463e+01,310.369566,21177.656724,19684.920294,3.914360
min,1.000000,1.674747e+09,14.000000,21.000000,21.000000,6.000000
25%,659.000000,1.674747e+09,42.000000,80.000000,80.000000,6.000000
50%,1317.000000,1.674747e+09,54.000000,8884.500000,6668.000000,6.000000
75%,1975.000000,1.674747e+09,82.000000,44171.500000,39861.500000,6.000000
max,2633.000000,1.674747e+09,2962.000000,62110.000000,62110.000000,17.000000


In [19]:
df_1.describe()

,frame_number,frame_time_epoch,frame_len,srcport,dstport,ip_proto
count,2335.000000,2.335000e+03,2335.000000,1225.000000,1225.000000,1138.000000
mean,1168.000000,1.674748e+09,112.679229,21132.053061,16402.749388,7.817223
std,674.200761,1.961275e+01,243.664405,20583.274522,18918.959190,4.086787
min,1.000000,1.674748e+09,42.000000,21.000000,21.000000,6.000000
25%,584.500000,1.674748e+09,42.000000,53.000000,53.000000,6.000000
50%,1168.000000,1.674748e+09,54.000000,8886.000000,6668.000000,6.000000
75%,1751.500000,1.674748e+09,74.000000,42723.000000,36261.000000,6.000000
max,2335.000000,1.674748e+09,1514.000000,62110.000000,62110.000000,17.000000


In [22]:
df[df["payload_utf8"].notnull()]

,frame_number,frame_time_epoch,frame_len,eth_src,device_name_src,eth_dst,device_name_dst,eth_type,ip_src,srcport,ip_dst,dstport,ip_proto,payload_utf8,action,attack
1,2,1.674747e+09,303,A8:03:2A:B1:35:60,NaN,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.80,51705.0,34.140.193.14,6022.0,6.0,,NaN,NaN
8,9,1.674747e+09,107,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.200,17089.0,52.59.155.15,443.0,6.0,,NaN,NaN
9,10,1.674747e+09,107,04:CF:4B:3B:00:58,NaN,60:01:94:C7:69:AC,Lampe2,0x0800,52.59.155.15,443.0,192.168.12.200,17089.0,6.0,,NaN,NaN
12,13,1.674747e+09,267,D8:F1:5B:D8:08:0C,LDR TUYA,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.113,13881.0,18.192.43.219,8886.0,6.0,,"""237.00""",NaN
17,18,1.674747e+09,123,04:CF:4B:3B:00:58,NaN,D8:F1:5B:D8:08:0C,LDR TUYA,0x0800,18.192.43.219,8886.0,192.168.12.113,13881.0,6.0,,"""237.00""",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,2609,1.674747e+09,256,04:CF:4B:3B:00:58,NaN,04:CF:4B:3B:00:38,NaN,0x0800,192.168.12.1,53.0,192.168.12.110,57981.0,17.0,&,NaN,NaN
2609,2610,1.674747e+09,89,A8:03:2A:B1:35:60,NaN,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.80,51705.0,34.140.193.14,6022.0,6.0,,NaN,NaN
2611,2612,1.674747e+09,85,04:CF:4B:3B:00:58,NaN,A8:03:2A:B1:35:60,NaN,0x0800,34.140.193.14,6022.0,192.168.12.80,51705.0,6.0,,NaN,NaN
2628,2629,1.674747e+09,89,A8:03:2A:B1:35:60,NaN,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.80,51705.0,34.140.193.14,6022.0,6.0,,NaN,NaN


In [24]:
df[df["device_name_src"] == "Lampe2"]

,frame_number,frame_time_epoch,frame_len,eth_src,device_name_src,eth_dst,device_name_dst,eth_type,ip_src,srcport,ip_dst,dstport,ip_proto,payload_utf8,action,attack
8,9,1.674747e+09,107,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.200,17089.0,52.59.155.15,443.0,6.0,,NaN,NaN
10,11,1.674747e+09,54,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:58,NaN,0x0800,192.168.12.200,17089.0,52.59.155.15,443.0,6.0,NaN,NaN,NaN
464,465,1.674747e+09,42,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:38,NaN,0x0806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1113,1114,1.674747e+09,42,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:58,NaN,0x0806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tcp_portscan
1544,1545,1.674747e+09,42,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:38,NaN,0x0806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1546,1547,1.674747e+09,70,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:38,NaN,0x0800,"192.168.12.200,192.168.12.110",53.0,"192.168.12.110,192.168.12.200",53.0,NaN,NaN,NaN,tcp_portscan
1551,1552,1.674747e+09,42,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:38,NaN,0x0806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tcp_portscan
1560,1561,1.674747e+09,54,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:38,NaN,0x0800,192.168.12.200,23.0,192.168.12.110,40275.0,6.0,NaN,NaN,tcp_portscan
1561,1562,1.674747e+09,54,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:38,NaN,0x0800,192.168.12.200,22.0,192.168.12.110,45943.0,6.0,NaN,NaN,tcp_portscan
1562,1563,1.674747e+09,54,60:01:94:C7:69:AC,Lampe2,04:CF:4B:3B:00:38,NaN,0x0800,192.168.12.200,24.0,192.168.12.110,33149.0,6.0,NaN,NaN,tcp_portscan


In [26]:
df["payload_utf8"].unique()

array([nan, '\x17\x03\x03',
       '{"device":"ALPHA","mac":"AC:0B:FB:D8:38:4E","sensor":"LDR TUYA","value":"237.00"}',
       '{"attack": "hostscan", "target": "192.168.12.1", "MAC": "04:cf:4b:3b:00:38", "type": "start", "time": 1674746658774}',
       '{"device":"ALPHA","mac":"AC:0B:FB:D8:38:4E","sensor":"LDR TUYA","value":"-661.00"}',
       '{"attack": "hostscan", "target": "192.168.12.1", "MAC": "04:cf:4b:3b:00:38", "type": "stop", "time": 1674746661932}',
       '{"attack": "tcp_portscan", "target": "192.168.12.1", "MAC": "04:CF:4B:3B:00:58", "type": "start", "time": 1674746662257}',
       '{"attack": "tcp_portscan", "target": "192.168.12.1", "MAC": "04:CF:4B:3B:00:58", "type": "stop", "time": 1674746663480}',
       '{"attack": "ftp_wordlist", "target": "192.168.12.1", "MAC": "04:CF:4B:3B:00:58", "type": "start", "time": 1674746663543}',
       '{"device":"ALPHA","mac":"AC:0B:FB:D8:38:4E","sensor":"LDR TUYA","value":"621.00"}',
       '{"attack": "ftp_wordlist", "target": "192.

In [27]:
df_1["payload_utf8"].unique()

array([nan, '\x17\x03\x03',
       '{"device":"ALPHA","mac":"AC:0B:FB:D8:38:4E","sensor":"LDR TUYA","value":"-571.00"}',
       '\x01\x01', '\x01', ',\x01', ',', '遀',
       '{"device":"ALPHA","mac":"AC:0B:FB:D8:38:4E","sensor":"LDR TUYA","value":"288.00"}',
       '{"attack": "hostscan", "target": "192.168.12.1", "MAC": "04:cf:4b:3b:00:38", "type": "start", "time": 1674747934040}',
       '{"device":"ALPHA","mac":"AC:0B:FB:D8:38:4E","sensor":"LDR TUYA","value":"-563.00"}',
       '{"attack": "hostscan", "target": "192.168.12.1", "MAC": "04:cf:4b:3b:00:38", "type": "stop", "time": 1674747938286}',
       '{"attack": "tcp_portscan", "target": "192.168.12.1", "MAC": "04:CF:4B:3B:00:58", "type": "start", "time": 1674747938440}',
       '{"attack": "tcp_portscan", "target": "192.168.12.1", "MAC": "04:CF:4B:3B:00:58", "type": "stop", "time": 1674747939598}',
       '{"attack": "ftp_wordlist", "target": "192.168.12.1", "MAC": "04:CF:4B:3B:00:58", "type": "start", "time": 1674747939655}',
   

In [28]:
df["device_name_src"].unique()

array([nan, 'LDR TUYA', 'Lampe2', 'GARAGE_DOOR', 'SmartSocket1'],
      dtype=object)

In [29]:
df_1["device_name_src"].unique()

array([nan, 'LDR TUYA', 'GARAGE_DOOR', 'SmartSocket1', 'Lampe2'],
      dtype=object)